**Removal of Similar Frame**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install vulture

In [ ]:
import cv2
import pathlib
import os
video_path='path to input video'
# capture = cv2.VideoCapture(video_path)
video_size_mb = os.path.getsize(pathlib.Path(video_path)) / (1024 * 1024)

print(f"Video size: {video_size_mb:.2f} MB")

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import cv2
import os

# Input and output paths
input_video_path = '/content/drive/MyDrive/Malayalam-Sita-Ramam-Full-screen.mp4'
output_video_path = '/content/output.mp4'

# Get initial video size
initial_size = os.path.getsize(input_video_path)

# Load video
cap = cv2.VideoCapture(input_video_path)

# Get video properties
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

**Pipeline for FPS Reduction**

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
if(fps<=15):
  fps=15
elif(fps>15 and fps<=30):
  fps=fps
else:
  fps=fps//2

In [ ]:
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frames = []
frames1=[]
initial_frame_count = 0
j=0
while True:
    ret, frame = cap.read()

    if not ret:
        break

    initial_frame_count += 1
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame_flat = gray_frame.flatten()

    if frames:
        prev_frame = frames[-1]
        similarity = cosine_similarity([gray_frame_flat], [prev_frame])
        if similarity >1.1:
            continue

    frames.append(gray_frame_flat)

    # Resize frame to lower resolution
    frame = cv2.resize(frame, (width, height))
    # frame = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
    frames1.append(frame)
    j+=1
    # cv2.imwrite('/content/dhoni/new/'+str(j)+'.jpeg',frame)
    out.write(frame)

cap.release()
out.release()

# Get final video size
final_size = os.path.getsize(output_video_path)

print(f"Initial video size: {initial_size / (1024 * 1024):.2f} MB")
print(f"Final video size: {final_size / (1024 * 1024):.2f} MB")
print(f"Initial frame count: {initial_frame_count}")
print(f"Final frame count: {len(frames)}")



**Extraction of faces from video using caffe model**

In [ ]:
import os
import cv2
import numpy as np

In [ ]:
%cd "/content"

/content


In [ ]:
!git clone "https://github.com/kb22/Create-Face-Data-from-Images"

In [ ]:
%cd "/content/Create-Face-Data-from-Images"

**Load Caffe Model Path**

In [ ]:
prototxt_path = '/content/Create-Face-Data-from-Images/model_data/deploy.prototxt'
caffemodel_path = '/content/Create-Face-Data-from-Images/model_data/weights.caffemodel'

# Read the model
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

**Create Directory to extract faces**

In [ ]:
if not os.path.exists('faces'):
	print("New directory created")
	os.makedirs('faces')

**Deepface Model Installation**

In [ ]:
!pip install deepface

In [ ]:
%cd "/content"
! git clone "https://github.com/serengil/deepface.git"
%cd deepface
!pip install -e .

In [ ]:
from deepface import DeepFace

**Loading of Haarcascade model**

In [ ]:
from google.colab.patches import cv2_imshow
body_classifier = cv2.CascadeClassifier('/content/haarcascade_fullbody.xml')

**Get video height and width**

In [ ]:
frames1=np.asarray(frames1)
(h, w) = frames1.shape[1:3]
print("Height of frame=",h)
print("Width of frame=",w)

**Pipeline for frame elimination**

**1 Character Verified**

In [ ]:
l=0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path="/content/001output.mp4"
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w,h))
# all frames of video stored in frames 1
for image in frames1:
  #image is one frame among frame1
  #extraction of face from image
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
  model.setInput(blob)
  detections = model.forward()
  flag=0
  flag1=0
  fme=image
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #detect body in frame and return their coordinates
  bodies = body_classifier.detectMultiScale(gray, 1.2, 3)
  if(len(bodies)!=0):
    flag1=1
  for i in range(0, detections.shape[2]):
    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")
    confidence = detections[0, 0, i, 2]
    # If confidence > 0.5, show box around face
    if (confidence > 0.5):
      l+=1
      img = image[startY:endY,startX:endX]
      print(img.shape)
      #img is the frame with face with some additioal attributes
      if img is img.shape[0] == 0 or img.shape[1] == 0:
        continue
      cv2.imwrite('/content/faces1/'+str(l)+'.jpeg', img)
      #for verification of extracted face with database
      result=DeepFace.find(img_path = '/content/faces1/'+str(l)+'.jpeg', db_path = "/content/drive/MyDrive/1_character",enforce_detection = False)
      # return the dataframe if it found
      print(result.columns)
      if(result[0].shape[0]==0):
         flag=1
  #this condition if there is human in frame and his face is also detected but not verified
  if(flag==0 and flag1==1):
    out.write(image)
  #this conditon if there is no face get detected in the image but there is presence of human in frame
  if(flag==1):
    out.write(image)
    print('!')
  #in other condition we eliminate the frame
out.release()

**Final Video Size**

In [ ]:
final_size = os.path.getsize('/content/001output.mp4')
print(f"Final video size: {final_size / (1024 * 1024):.2f} MB")

**2 character elimination**

In [ ]:

l=0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path="/content/002output.mp4"
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w,h))
# all frames of video stored in frames 1
for image in frames1:
  #image is one frame among frame1
  #extraction of face from image
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
  model.setInput(blob)
  detections = model.forward()
  flag=0
  flag1=0
  fme=image
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #detect body in frame and return their coordinates
  bodies = body_classifier.detectMultiScale(gray, 1.2, 3)
  if(len(bodies)!=0):
    flag1=1
  for i in range(0, detections.shape[2]):
    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")
    confidence = detections[0, 0, i, 2]
    # If confidence > 0.5, show box around face
    if (confidence > 0.5):
      l+=1
      img = image[startY:endY,startX:endX]
      print(img.shape)
      #img is the frame with face with some additioal attributes
      if img is img.shape[0] == 0 or img.shape[1] == 0:
        continue
      cv2.imwrite('/content/faces2/'+str(l)+'.jpeg', img)
      #for verification of extracted face with database
      result=DeepFace.find(img_path = '/content/faces2/'+str(l)+'.jpeg', db_path = "/content/drive/MyDrive/2_character",enforce_detection = False)
      # return the dataframe if it found
      if(result[0].shape[0]==0):
         flag=1
  #this condition if there is human in frame and his face is also detected but not verified
  if(flag==0 and flag1==1):
    out.write(image)
  #this conditon if there is no face get detected in the image but there is presence of human in frame
  if(flag==1):
    out.write(image)
    print('!')
  #in other condition we eliminate the frame
out.release()

**Final Video Size**

In [ ]:
final_size = os.path.getsize('/content/002output.mp4')
print(f"Final video size: {final_size / (1024 * 1024):.2f} MB")